In [ ]:
import numpy as np
import pickle
import os

seed = 2023

In [ ]:
import torch

# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
from nlpsig_networks.scripts.lstm_baseline_functions import (
    lstm_hyperparameter_search,
    obtain_path,
)

In [ ]:
output_dir = "client_talk_type_output"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

## AnnoMI

In [ ]:
%run ../load_anno_mi.py

In [ ]:
anno_mi.head()

In [ ]:
with open("../anno_mi_sbert.pkl", "rb") as f:
    sbert_embeddings = pickle.load(f)

sbert_embeddings.shape

# Baseline: LSTM classification

In [ ]:
num_epochs = 100
hidden_dim_sizes = [200, 300, 400]
num_layers = 1
bidirectional = True
dropout_rates = [0.1]
learning_rates = [1e-3, 5e-4, 1e-4]
seeds = [1, 12, 123]
loss = "focal"
gamma = 2
validation_metric = "f1"
patience = 3

In [ ]:
kwargs = {
    "num_epochs": num_epochs,
    "df": anno_mi,
    "id_column": "transcript_id",
    "label_column": "client_talk_type",
    "embeddings": sbert_embeddings,
    "y_data": y_data_client,
    "output_dim": output_dim_client,
    "hidden_dim_sizes": hidden_dim_sizes,
    "num_layers": num_layers,
    "bidirectional": bidirectional,
    "dropout_rates": dropout_rates,
    "learning_rates": learning_rates,
    "seeds": seeds,
    "loss": loss,
    "gamma": gamma,
    "device": device,
    "path_indices": client_index,
    "split_ids": client_transcript_id,
    "k_fold": True,
    "patience": patience,
    "validation_metric": validation_metric,
    "verbose": False,
}

## history_length=5

In [ ]:
size = 5

In [ ]:
(
    bilstm_history_5_kfold,
    best_bilstm_history_5_kfold,
    _,
    __,
) = lstm_hyperparameter_search(
    history_lengths=[size],
    results_output=f"{output_dir}/lstm_history_{size}_focal_{gamma}_kfold.csv",
    **kwargs,
)

In [ ]:
bilstm_history_5_kfold.groupby(["hidden_dim", "dropout_rate", "learning_rate"]).mean()

In [ ]:
best_bilstm_history_5_kfold

In [ ]:
best_bilstm_history_5_kfold["f1"].mean()

In [ ]:
best_bilstm_history_5_kfold["precision"].mean()

In [ ]:
best_bilstm_history_5_kfold["recall"].mean()

In [ ]:
np.stack(best_bilstm_history_5_kfold["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_bilstm_history_5_kfold["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_bilstm_history_5_kfold["recall_scores"]).mean(axis=0)

## history_length=11

In [ ]:
size = 11

In [ ]:
(
    bilstm_history_11_kfold,
    best_bilstm_history_11_kfold,
    _,
    __,
) = lstm_hyperparameter_search(
    history_lengths=[size],
    results_output=f"{output_dir}/lstm_history_{size}_focal_{gamma}_kfold.csv",
    **kwargs,
)

In [ ]:
bilstm_history_11_kfold.groupby(["hidden_dim", "dropout_rate", "learning_rate"]).mean()

In [ ]:
best_bilstm_history_11_kfold

In [ ]:
best_bilstm_history_11_kfold["f1"].mean()

In [ ]:
best_bilstm_history_11_kfold["precision"].mean()

In [ ]:
best_bilstm_history_11_kfold["recall"].mean()

In [ ]:
np.stack(best_bilstm_history_11_kfold["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_bilstm_history_11_kfold["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_bilstm_history_11_kfold["recall_scores"]).mean(axis=0)

## history_length=20

In [ ]:
size = 20

In [ ]:
(
    bilstm_history_20_kfold,
    best_bilstm_history_20_kfold,
    _,
    __,
) = lstm_hyperparameter_search(
    history_lengths=[size],
    results_output=f"{output_dir}/lstm_history_{size}_focal_{gamma}_kfold.csv",
    **kwargs,
)

In [ ]:
bilstm_history_20_kfold.groupby(["hidden_dim", "dropout_rate", "learning_rate"]).mean()

In [ ]:
best_bilstm_history_20_kfold

In [ ]:
best_bilstm_history_20_kfold["f1"].mean()

In [ ]:
best_bilstm_history_20_kfold["precision"].mean()

In [ ]:
best_bilstm_history_20_kfold["recall"].mean()

In [ ]:
np.stack(best_bilstm_history_20_kfold["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_bilstm_history_20_kfold["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_bilstm_history_20_kfold["recall_scores"]).mean(axis=0)

## history_length=35

In [ ]:
size = 35

In [ ]:
(
    bilstm_history_35_kfold,
    best_bilstm_history_35_kfold,
    _,
    __,
) = lstm_hyperparameter_search(
    history_lengths=[size],
    results_output=f"{output_dir}/lstm_history_{size}_focal_{gamma}_kfold.csv",
    **kwargs,
)

In [ ]:
bilstm_history_35_kfold.groupby(["hidden_dim", "dropout_rate", "learning_rate"]).mean()

In [ ]:
best_bilstm_history_35_kfold

In [ ]:
best_bilstm_history_35_kfold["f1"].mean()

In [ ]:
best_bilstm_history_35_kfold["precision"].mean()

In [ ]:
best_bilstm_history_35_kfold["recall"].mean()

In [ ]:
np.stack(best_bilstm_history_35_kfold["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_bilstm_history_35_kfold["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_bilstm_history_35_kfold["recall_scores"]).mean(axis=0)

## history_length=80

In [ ]:
size = 80

In [ ]:
(
    bilstm_history_80_kfold,
    best_bilstm_history_80_kfold,
    _,
    __,
) = lstm_hyperparameter_search(
    history_lengths=[size],
    results_output=f"{output_dir}/lstm_history_{size}_focal_{gamma}_kfold.csv",
    **kwargs,
)

In [ ]:
bilstm_history_80_kfold.groupby(["hidden_dim", "dropout_rate", "learning_rate"]).mean()

In [ ]:
best_bilstm_history_80_kfold

In [ ]:
best_bilstm_history_80_kfold["f1"].mean()

In [ ]:
best_bilstm_history_80_kfold["precision"].mean()

In [ ]:
best_bilstm_history_80_kfold["recall"].mean()

In [ ]:
np.stack(best_bilstm_history_80_kfold["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_bilstm_history_80_kfold["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_bilstm_history_80_kfold["recall_scores"]).mean(axis=0)

## history_length=110

In [ ]:
size = 110

In [ ]:
(
    bilstm_history_110_kfold,
    best_bilstm_history_110_kfold,
    _,
    __,
) = lstm_hyperparameter_search(
    history_lengths=[size],
    results_output=f"{output_dir}/lstm_history_{size}_focal_{gamma}_kfold.csv",
    **kwargs,
)

In [ ]:
bilstm_history_110_kfold.groupby(["hidden_dim", "dropout_rate", "learning_rate"]).mean()

In [ ]:
best_bilstm_history_110_kfold

In [ ]:
best_bilstm_history_110_kfold["f1"].mean()

In [ ]:
best_bilstm_history_110_kfold["precision"].mean()

In [ ]:
best_bilstm_history_110_kfold["recall"].mean()

In [ ]:
np.stack(best_bilstm_history_110_kfold["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_bilstm_history_110_kfold["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_bilstm_history_110_kfold["recall_scores"]).mean(axis=0)